APC Summarizer

Written by Vivek

Started on Feb 6th, 2017

This program creates summary counts of ridership for any given day by ons using APC or automated passenger count data.

- It gathers the raw data from the CAD/AVL ACS database from the table apc_correlated
- In the Panadas python code I do group by's to determine counts per route and direction, then how many total trips are sampled for the given day per route and direction.
- Then using a csv derived from GTFSDB for the April GTFS, I figure out how many total trips have happended for that route and direction.  Right now that is just in CSV form as the query is run elsewhere.
- select pst.arrival_time, pst.trip_id, pst.stop_sequence, pt.route_id, pt.service_id, pt.block_id, pt.direction_id, pt.shape_id, pt.trip_headsign from public.stop_times pst left join public.trips pt ON (pst.trip_id = pt.trip_id) where pst.stop_sequence = '1' and pt.service_id = 'Weekdays' order by pt.route_id, pst.arrival_time;

Then the three tables are joined, a little bit of math to determine Projected Ridership:  
((Ridership for a trip by route by direction by time)/(Total number of sampled trips by route by direction by time))*(Total trips run by route by direction by time) = Projected Ridership for a trip by route by time group.

Then pivot to make it easier to read.  Then reordered columns.

Limitations:
The current Total Trips only works for days in the first quarter of 2016 with no special trips since it only pulls those gtfs files.

How to get this to work:
Use a windows machine on the network.
I prefer installing anaconda, then use a python 3.5 environment.  Use the launch gui to install the necessary packages Pandas and pypyodbc (or another way to install pypyodbc)
Open up a Jupyter notebook and run!

In [1]:
#Pull data from the apc database, use current_route_id.  df gets the raw data.
import pypyodbc
import pandas as pd
import connect
#connect.connect_string= 'uid=abcd;pwd=pwdsrule'
connection = pypyodbc.connect("Driver={SQL Server};Server=ELTDBPRD\ELTDBPRD;Database=ACS_13;" + connect.connect_string)
start_date = '2016/03/01'
end_date = '2016/03/25'
sql='''SELECT [apc_date_time],[start_trip_time],[vehicle_id],[route_id],[block_id],[current_route_id],[apc].[direction_code_id], [dc].[direction_description], [variation],[bs_id],[run_id],[operator_id],[dwell_time],[ons],[offs],[max_load],[position_source],[quality_indicator],[raw_on],[raw_off],[raw_max_load],[booking_num],[ext_trip_id],[headsign_route],[sched_time],[tp_id],[booking_id],[booking_start_date],[veh_lat],[veh_long],[insert_date_time] 
  FROM [ACS_13].[dbo].[apc_correlated] apc 
  left join [ACS_13].[dbo].[direction_codes] dc
  ON apc.direction_code_id=dc.direction_code_id
    where ext_trip_id != 0 and current_route_id != 0 and 
    apc_date_time between '2016/03/01' and '2016/03/04' order by apc_date_time ASC;'''

#Monthly Summary String
    #(apc_date_time between '2016/03/01' and '2016/03/04' or apc_date_time between '2016/03/08' and '2016/03/11' or apc_date_time between '2016/03/15' and '2016/03/18' or apc_date_time between '2016/03/22' and '2016/03/25') order by apc_date_time ASC;'''
#apc_date_time between '%s' and '%s';''' % (start_date, end_date)

df = pd.read_sql(sql, connection)

del df['route_id']
df= df.rename(columns = {'current_route_id':'route_id'})

In [2]:
df

,apc_date_time,start_trip_time,vehicle_id,block_id,route_id,direction_code_id,direction_description,variation,bs_id,run_id,...,booking_num,ext_trip_id,headsign_route,sched_time,tp_id,booking_id,booking_start_date,veh_lat,veh_long,insert_date_time
0,2016-03-01 00:00:55,1900-01-01 00:07:00,175,2312,23,33,EAST,EB04A_,5637,5130,...,4,1923117,23,2016-02-24 00:07:00,127,JA16 005,2016-02-24,37.322521,-122.044425,2016-03-01 01:12:57
1,2016-03-01 00:01:15,1900-01-01 00:07:00,175,2312,23,33,EAST,EB04A_,0,5130,...,4,1923117,23,2016-03-01 00:01:15,0,JA16 005,2016-02-24,37.322521,-122.044433,2016-03-01 01:12:57
2,2016-03-01 00:01:31,1900-01-01 23:40:00,193,2252,22,33,EAST,EB04A_,355,2137,...,4,1922050,22,2016-02-24 00:01:00,146,JA16 005,2016-02-24,37.385128,-122.082901,2016-03-01 01:53:02
3,2016-03-01 00:02:01,1900-01-01 23:53:00,2057,7073,200,34,NORTH,NB01__,2526,4186,...,4,1935530,200,2016-02-24 00:01:00,425,JA16 005,2016-02-24,37.403484,-121.980690,2016-03-01 01:02:52
4,2016-03-01 00:02:18,1900-01-01 23:45:00,170,2316,23,36,WEST,WB07__,3264,2316,...,4,1923190,23,2016-02-24 00:05:00,8409,JA16 005,2016-02-24,37.336109,-121.888999,2016-03-01 01:02:51
5,2016-03-01 00:02:18,1900-01-01 23:25:00,1031,133,181,35,SOUTH,SB04__,5513,5278,...,4,1934919,181,2016-02-24 00:02:00,8409,JA16 005,2016-02-24,37.336193,-121.889144,2016-03-01 00:22:39
6,2016-03-01 00:02:51,1900-01-01 23:30:00,172,2302,22,36,WEST,WB03A_,440,2302,...,4,1922140,22,2016-02-23 23:56:00,8419,JA16 005,2016-02-24,37.336383,-121.889854,2016-03-01 02:23:04
7,2016-03-01 00:02:56,1900-01-01 23:51:00,2065,2511,25,33,EAST,EB03__,744,2511,...,4,1923995,25,2016-02-24 00:03:00,466,JA16 005,2016-02-24,37.308666,-121.900650,2016-03-01 00:42:47
8,2016-03-01 00:03:29,1900-01-01 23:53:00,2057,7073,200,34,NORTH,NB01__,2365,4186,...,4,1935530,200,2016-03-01 00:03:29,0,JA16 005,2016-02-24,37.403438,-121.989067,2016-03-01 01:02:52
9,2016-03-01 00:04:14,1900-01-01 23:30:00,172,2302,22,36,WEST,WB03A_,441,2302,...,4,1922140,22,2016-03-01 00:04:14,0,JA16 005,2016-02-24,37.335453,-121.892105,2016-03-01 02:23:04


In [3]:
#Create a new column that groups times by slots 5-9am, etc.
df.loc[(df['start_trip_time'].dt.hour >= 5) & (df['start_trip_time'].dt.hour < 9), 'Time_Group'] = '5A-9A'
df.loc[(df['start_trip_time'].dt.hour >= 9) & (df['start_trip_time'].dt.hour < 15), 'Time_Group'] = '9A-3P'
df.loc[(df['start_trip_time'].dt.hour >= 15) & (df['start_trip_time'].dt.hour < 19), 'Time_Group'] = '3P-7P'
df.loc[df['start_trip_time'].dt.hour >= 19, 'Time_Group'] = '7P-5A'
df.loc[df['start_trip_time'].dt.hour < 5, 'Time_Group'] = '7P-5A'

df['dayofyear']= df.apc_date_time.dt.dayofyear
#Group by route, time group, and direction.  Summarized df gives the raw ons.
summarized_df = df.groupby(['Time_Group','route_id','direction_description'])['ons'].sum()
summarized_df = summarized_df.reset_index()
summarized_df = summarized_df.rename(columns = {'ons':'tot_ons'})

#nunique table contains the number of unique sampled trips per day, then sum up the days.
nunique_df = df.groupby(['Time_Group','route_id','direction_description', 'dayofyear'])['ext_trip_id'].nunique()
nunique_df = nunique_df.reset_index().groupby(['Time_Group','route_id', 'direction_description']) ['ext_trip_id'].sum().reset_index()
#nunique_df = nunique_df.reset_index()
nunique_df = nunique_df.rename(columns = {'ext_trip_id':'samp_trips'})

In [4]:
#Grab a csv derived from gtfs that tells you when each trip starts
gtfs=pd.read_csv('start_times_trips_gtfs_march_2016.csv')
#A table that goes from cardinal direction (east, west, etc.) to direction_id (inbound, outbound) so you may join apc to gtfs.
line_table=pd.read_csv('line_table.csv')

#http://pandas.pydata.org/pandas-docs/stable/merging.html
gtfs = pd.merge(gtfs, line_table, how='left', left_on=['route_id','direction_id'], right_on=['lineabbr', 'DirNum'])
gtfs['arrival_time']=gtfs['arrival_time'].str.replace('^24:','00:')
gtfs['arrival_time']=gtfs['arrival_time'].apply(pd.to_datetime,format='%H:%M:%S')

#Reground, similar to above
gtfs.loc[(gtfs['arrival_time'].dt.hour >= 5) & (gtfs['arrival_time'].dt.hour < 9), 'Time_Group'] = '5A-9A'
gtfs.loc[(gtfs['arrival_time'].dt.hour >= 9) & (gtfs['arrival_time'].dt.hour < 15), 'Time_Group'] = '9A-3P'
gtfs.loc[(gtfs['arrival_time'].dt.hour >= 15) & (gtfs['arrival_time'].dt.hour < 19), 'Time_Group'] = '3P-7P'
gtfs.loc[gtfs['arrival_time'].dt.hour >= 19, 'Time_Group'] = '7P-5A'
gtfs.loc[gtfs['arrival_time'].dt.hour < 5, 'Time_Group'] = '7P-5A'

In [5]:
#http://wesmckinney.com/blog/groupby-fu-improvements-in-grouping-and-aggregating-data-in-pandas/
#summarized_gtfs tells you the total number of trips run
summarized_gtfs = gtfs.groupby(['Time_Group','route_id','direction_id','DirectionName'])['trip_id'].count()
summarized_gtfs = summarized_gtfs.reset_index()
summarized_gtfs= summarized_gtfs.rename(columns = {'trip_id':'tot_trips', 'DirectionName':'direction_description'})
#summarized_gtfs['Time_Group'].value_counts()

In [6]:
summarized_gtfs['route_id'].unique()
summarized_gtfs['route_id'].unique()
counts = pd.merge(summarized_df, nunique_df, how='outer', on =['Time_Group','route_id','direction_description'])
counts = pd.merge(counts, summarized_gtfs, how='outer', on =['Time_Group','route_id','direction_description'])

In [7]:
#Join all three tables together
counts = pd.merge(summarized_df, nunique_df, how='outer', on =['Time_Group','route_id','direction_description'])
counts = pd.merge(counts, summarized_gtfs, how='outer', on =['Time_Group','route_id','direction_description'])
#Do the math described in the intro to get Projected trips per day
counts['total_exp_ons_day']=(counts['tot_ons']/counts['samp_trips'])*counts['tot_trips']
# del counts['tot_ons']
# del counts['samp_trips']
# del counts['tot_trips']
del counts['direction_id']

In [8]:
#Pivot and reorder to get more readable tables reorder tables
counts_full = counts.pivot_table(index= ['route_id', 'direction_description', ], columns='Time_Group', values=['tot_ons','samp_trips','tot_trips', 'total_exp_ons_day'])
#counts_summ = counts.pivot_table(index= ['route_id', 'direction_description', ], columns='Time_Group', values=['total_exp_ons_day'])
counts_summ = counts.pivot_table(index= ['route_id', 'direction_description', ], columns='Time_Group', values=['total_exp_ons_day'], aggfunc = sum, margins=True, margins_name = 'Total Proj')
cols = counts_summ.columns.tolist()
cols = [cols[1], cols[3], cols[0], cols[2], cols[4]]
counts_summ = counts_summ[cols]
cols = counts_full.columns.tolist()
cols = [cols[1], cols[3], cols[0], cols[2], cols[1+4], cols[3+4], cols[0+4], cols[2+4], cols[1+8], cols[3+8], cols[0+8], cols[2+8], cols[1+12], cols[3+12], cols[0+12], cols[2+12]]
counts_full = counts_full[cols]

In [9]:
#Output daily summaries
counts_summ.to_csv('summaries/' + start_date.replace('/','_') + '_' + end_date.replace('/','_') + '_' + 'counts_summ.csv')
counts_full.to_csv('summaries/' + start_date.replace('/','_') + '_' + end_date.replace('/','_') + '_' + 'counts_full.csv')

In [10]:
counts_full

tot_ons                         samp_trips  \
Time_Group                       5A-9A   9A-3P   3P-7P   7P-5A      5A-9A   
route_id direction_description                                              
10       EAST                    356.0   634.0   490.0   186.0       44.0   
         WEST                    250.0   465.0   348.0   223.0       38.0   
13       NORTH                    68.0    22.0    15.0     NaN        4.0   
         SOUTH                    51.0    84.0    38.0     4.0        4.0   
14       NORTH                    36.0   164.0    17.0     NaN        6.0   
         SOUTH                     9.0    53.0    11.0     NaN        6.0   
16       NORTH                    51.0     4.0     6.0     NaN        8.0   
         SOUTH                    18.0     2.0    99.0     NaN        9.0   
17       NORTH                    13.0    42.0     NaN     NaN        6.0   
         SOUTH                     4.0    78.0     9.0     NaN        3.0   
18       NORTH                    16.0   134.0    40.0     5.0        9.0   
         SOUTH                   136.0    99.0     9.0     7.0       12.0   
19       NORTH                    23.0   128.0    64.0     NaN        9.0   
         SOUTH                    73.0   165.0    26.0     NaN       11.0   
22       EAST                   1396.0  2724.0  1487.0  1394.0       25.0   
         WEST                   1522.0  3098.0  1239.0  1587.0       17.0   
23       EAST                   1713.0  5402.0  2967.0  1176.0       49.0   
         WEST                   3055.0  5117.0  2159.0   568.0       58.0   
25       EAST                   1006.0  2770.0  1454.0   292.0       28.0   
         WEST                   1597.0  2282.0   991.0   179.0       38.0   
26       EAST                    679.0  1123.0   723.0   156.0       13.0   
         WEST                    775.0  1059.0   656.0   185.0       13.0   
27       EAST                     86.0   190.0   146.0    23.0        7.0   
         WEST                    123.0   210.0   132.0     9.0        7.0   
31       NORTH                    42.0   205.0    75.0    54.0        7.0   
         SOUTH                   176.0   167.0    52.0    67.0       10.0   
32       EAST                    388.0   675.0   654.0    31.0       20.0   
         WEST                    597.0   581.0   355.0    38.0       20.0   
34       EAST                      NaN   134.0     NaN     NaN        NaN   
         WEST                      NaN   104.0     NaN     NaN        NaN   
...                                ...     ...     ...     ...        ...   
323      EAST                    314.0  1323.0  1135.0   250.0       20.0   
         WEST                   1011.0  1796.0   738.0   238.0       32.0   
328      NORTH                    59.0     NaN     NaN     NaN        3.0   
         SOUTH                     NaN     NaN    69.0     NaN        NaN   
330      NORTH                    84.0     NaN     NaN     NaN        4.0   
         SOUTH                     NaN     NaN   121.0     NaN        NaN   
522      EAST                   1848.0  4107.0  2825.0   836.0       45.0   
         WEST                   3309.0  3986.0  2430.0   855.0       46.0   
822      NORTH                     NaN     NaN     NaN     NaN        NaN   
         SOUTH                     NaN     NaN     NaN     NaN        NaN   
823      EAST                      NaN     NaN     NaN     NaN        NaN   
         WEST                      NaN     NaN     NaN     NaN        NaN   
824      EAST                      NaN     NaN     NaN     NaN        NaN   
         WEST                      NaN     NaN     NaN     NaN        NaN   
825      EAST                      NaN     NaN     NaN     NaN        NaN   
         WEST                      NaN     NaN     NaN     NaN        NaN   
826      EAST                      NaN     NaN     NaN     NaN        NaN   
         WEST                      NaN     NaN     NaN     NaN        NaN   
827      NORTH             

In [11]:
counts_summ

total_exp_ons_day                \
Time_Group                                   5A-9A         9A-3P   
route_id   direction_description                                   
10         EAST                         210.363636    411.243243   
           WEST                         164.473684    328.235294   
13         NORTH                         34.000000     22.000000   
           SOUTH                         38.250000     84.000000   
14         NORTH                         12.000000     59.636364   
           SOUTH                          3.000000     35.333333   
16         NORTH                         31.875000      4.000000   
           SOUTH                          6.000000      2.000000   
17         NORTH                          4.333333     25.200000   
           SOUTH                          1.333333     29.250000   
18         NORTH                          7.111111     71.466667   
           SOUTH                         45.333333     52.800000   
19         NORTH                          7.666667     48.761905   
           SOUTH                         33.181818     61.875000   
22         EAST                        1228.480000   2641.454545   
           WEST                        1880.117647   2816.363636   
23         EAST                         629.265306   1840.241758   
           WEST                        1053.448276   1744.431818   
25         EAST                         646.714286   1784.067797   
           WEST                         966.605263   1563.592593   
26         EAST                         417.846154    842.250000   
           WEST                         655.769231    741.300000   
27         EAST                          73.714286    138.181818   
           WEST                         105.428571    152.727273   
31         NORTH                         36.000000    175.714286   
           SOUTH                        123.200000    143.142857   
32         EAST                         135.800000    225.000000   
           WEST                         208.950000    209.805556   
34         EAST                                NaN     52.111111   
           WEST                                NaN     34.666667   
...                                            ...           ...   
323        WEST                         347.531250    624.695652   
328        NORTH                         59.000000           NaN   
           SOUTH                               NaN           NaN   
330        NORTH                        126.000000           NaN   
           SOUTH                               NaN           NaN   
522        EAST                         616.000000   1408.114286   
           WEST                        1150.956522   1368.328358   
822        NORTH                               NaN           NaN   
           SOUTH                               NaN           NaN   
823        EAST                                NaN           NaN   
           WEST                                NaN           NaN   
824        EAST                                NaN           NaN   
           WEST                                NaN           NaN   
825        EAST                                NaN           NaN   
           WEST                                NaN           NaN   
826        EAST                                NaN           NaN   
           WEST                                NaN           NaN   
827        NORTH                               NaN           NaN   
           SOUTH                               NaN           NaN   
828        EAST                                NaN           NaN   
           WEST                                NaN           NaN   
831        EAST                                NaN           NaN   
           WEST                                NaN           NaN   
900        NORTH                               NaN           NaN   
           SOUTH                               NaN           NaN   
901        NORTH     